In [65]:
import pandas as pd
import warnings
import requests, zipfile, io, os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

In [66]:
# download file from url, extract it and store it in the dest_folder
def download_and_extract(url, dest_folder):
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(dest_folder)

In [ ]:
# define folders and urls
healthy_path = './healthy'
healthy_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewHealthy/Signal.zip'
parkinson_path = './parkinson'
parkinson_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewPatients/Signal.zip'

In [ ]:
download_and_extract(healthy_url, healthy_path)

In [ ]:
download_and_extract(parkinson_url, parkinson_path)

In [ ]:
# actual downloaded signals
healthy_signals = f'{healthy_path}/Signal/'
parkinson_signals = f'{parkinson_path}/Signal/'

In [ ]:
# define Input and labels (0 = healthy, parkinson = 1)
X = []
y = []

In [ ]:
# create dataset with sequence length 500

# healthy signals
for filename in os.listdir(healthy_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(healthy_signals + filename, sep='\s+', header=None, comment='#', nrows=3000) 
    X.append(df.to_numpy())
    y.append(0)

# parkinson signals
for filename in os.listdir(parkinson_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(parkinson_signals + filename, sep='\s+', header=None, comment='#', nrows=3000) 
    X.append(df.to_numpy())
    y.append(1)


In [ ]:
# convert lists to np arrays
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape

(792, 3000, 6)

In [ ]:
y.shape

(792,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_test.shape

(159, 3000, 6)

In [ ]:





model = keras.models.Sequential([
keras.layers.LSTM(128, return_sequences=True,  input_shape=(X_train.shape[1], 6)),
keras.layers.BatchNormalization( axis=-1,
    momentum=0.99,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",),
keras.layers.LSTM(128),
keras.layers.BatchNormalization( axis=-1,
    momentum=0.99,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",),
keras.layers.Dense(1, activation="sigmoid")
])


In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam",
metrics=["accuracy"])


In [61]:
model.fit(X_train, y_train, epochs = 1000, batch_size = 32)

Epoch 1/1000
20/20 [==============================] - 14s 709ms/step - loss: 0.8162 - accuracy: 0.5482
Epoch 2/1000
20/20 [==============================] - 14s 711ms/step - loss: 0.6559 - accuracy: 0.6035
Epoch 3/1000
20/20 [==============================] - 14s 710ms/step - loss: 0.6293 - accuracy: 0.6556
Epoch 4/1000
20/20 [==============================] - 14s 711ms/step - loss: 0.6061 - accuracy: 0.6746
Epoch 5/1000
20/20 [==============================] - 14s 712ms/step - loss: 0.6108 - accuracy: 0.6682
Epoch 6/1000
20/20 [==============================] - 14s 711ms/step - loss: 0.5810 - accuracy: 0.7014
Epoch 7/1000
20/20 [==============================] - 14s 710ms/step - loss: 0.5824 - accuracy: 0.6872
Epoch 8/1000
20/20 [==============================] - 14s 711ms/step - loss: 0.5690 - accuracy: 0.6888
Epoch 9/1000
20/20 [==============================] - 14s 709ms/step - loss: 0.5959 - accuracy: 0.6635
Epoch 10/1000
20/20 [==============================] - 14s 711ms/step - l

In [64]:
model.evaluate(X_test, y_test)


5/5 [==============================] - 1s 215ms/step - loss: 1.7625 - accuracy: 0.6667


[1.7625329494476318, 0.6666666865348816]